In [ ]:
##some code are adapted from:
##https://www.kaggle.com/code/sakethyalamanchili/machine-learning-for-bulldozer-prices
##https://www.kaggle.com/code/deryaumutkulal/bulldozer-price-prediction/notebook

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df=pd.read_csv('/content/Train.csv',low_memory=False,parse_dates=['saledate'])

In [ ]:
df.info()
df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32333 entries, 0 to 32332
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   32333 non-null  int64         
 1   SalePrice                 32333 non-null  int64         
 2   MachineID                 32333 non-null  int64         
 3   ModelID                   32333 non-null  int64         
 4   datasource                32333 non-null  int64         
 5   auctioneerID              32333 non-null  int64         
 6   YearMade                  32333 non-null  int64         
 7   MachineHoursCurrentMeter  24203 non-null  float64       
 8   UsageBand                 21277 non-null  object        
 9   saledate                  32333 non-null  datetime64[ns]
 10  fiModelDesc               32333 non-null  object        
 11  fiBaseModel               32333 non-null  object        
 12  fiSecondaryDesc   

,count,mean,min,25%,50%,75%,max,std
SalesID,32333.0,1199993.19927,1139246.0,1166490.0,1193352.0,1250504.0,1265220.0,39884.480554
SalePrice,32333.0,28809.013454,4750.0,12500.0,19500.0,37000.0,142000.0,23116.145205
MachineID,32333.0,971753.005258,323.0,1001026.0,1035584.0,1083840.0,1558273.0,344273.700997
ModelID,32333.0,7585.973773,28.0,3170.0,4665.0,9562.0,37198.0,6862.997914
datasource,32333.0,123.843813,121.0,121.0,121.0,132.0,132.0,4.816159
auctioneerID,32333.0,4.954783,1.0,3.0,3.0,3.0,99.0,11.215048
YearMade,32333.0,1842.234992,1000.0,1981.0,1998.0,2004.0,2010.0,360.254791
MachineHoursCurrentMeter,24203.0,4624.726108,0.0,934.0,2627.0,6417.5,220893.0,6038.939594
saledate,32333,2005-09-22 06:39:50.944236544,1989-01-31 00:00:00,2004-06-03 00:00:00,2008-05-01 00:00:00,2010-04-08 00:00:00,2011-12-29 00:00:00,NaN


In [ ]:
df.isnull()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32328,False,False,False,False,False,False,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True
32329,False,False,False,False,False,False,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True
32330,False,False,False,False,False,False,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True
32331,False,False,False,False,False,False,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
df['month']=df.saledate.dt.month
df['day']=df.saledate.dt.day
df['year']=df.saledate.dt.year
df.drop(['saledate'],axis=1,inplace=True)

In [ ]:
for label, content in df.items():
    if pd.api.types.is_object_dtype(content):
        df[label] = content.astype('category').cat.as_ordered()


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32333 entries, 0 to 32332
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   SalesID                   32333 non-null  int64   
 1   SalePrice                 32333 non-null  int64   
 2   MachineID                 32333 non-null  int64   
 3   ModelID                   32333 non-null  int64   
 4   datasource                32333 non-null  int64   
 5   auctioneerID              32333 non-null  int64   
 6   YearMade                  32333 non-null  int64   
 7   MachineHoursCurrentMeter  24203 non-null  float64 
 8   UsageBand                 21277 non-null  category
 9   fiModelDesc               32333 non-null  category
 10  fiBaseModel               32333 non-null  category
 11  fiSecondaryDesc           19905 non-null  category
 12  fiModelSeries             3823 non-null   category
 13  fiModelDescriptor         6467 non-null   cate

In [ ]:
df['state'].cat.categories
df['state'].cat.codes

0         0
1        32
2        31
3        43
4        31
         ..
32328     8
32329    43
32330     4
32331     4
32332     8
Length: 32333, dtype: int8

In [ ]:
df_train = df[df['year'] != 2009]
df_val = df[df['year'] == 2009]

len(df_train), len(df_val)

(27735, 4598)

In [ ]:
df.isnull().sum()

SalesID                         0
SalePrice                       0
MachineID                       0
ModelID                         0
datasource                      0
auctioneerID                    0
YearMade                        0
MachineHoursCurrentMeter     8130
UsageBand                   11056
fiModelDesc                     0
fiBaseModel                     0
fiSecondaryDesc             12428
fiModelSeries               28510
fiModelDescriptor           25866
ProductSize                 19361
fiProductClassDesc              0
state                           0
ProductGroup                    0
ProductGroupDesc                0
Drive_System                21188
Enclosure                      18
Forks                       13517
Pad_Type                    22658
Ride_Control                17850
Stick                       22658
Transmission                16456
Turbocharged                22658
Blade_Extension             30866
Blade_Width                 30866
Enclosure_Type

In [ ]:
for label, content in df_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():            df_train[label] = content.fillna(content.median())

<ipython-input-65-338e254415b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if pd.isnull(content).sum():            df_train[label] = content.fillna(content.median())


In [ ]:
for label, content in df_val.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_val[label] = content.fillna(content.median())

<ipython-input-66-891ca5b1824f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[label] = content.fillna(content.median())


In [ ]:
df['ProductGroup'].unique()

['WL', 'SSL', 'TEX', 'BL', 'TTT', 'MG']
Categories (6, object): ['BL' < 'MG' < 'SSL' < 'TEX' < 'TTT' < 'WL']

In [ ]:
df_train['age'] = df_train['year'] - df_train['YearMade']
df_val['age'] = df_val['year'] - df_val['YearMade']

<ipython-input-68-b8c537eba1a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['age'] = df_train['year'] - df_train['YearMade']
<ipython-input-68-b8c537eba1a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['age'] = df_val['year'] - df_val['YearMade']


In [ ]:
df_train.head().T

,0,1,2,3,5
SalesID,1139246,1139248,1139249,1139251,1139255
SalePrice,66000,57000,10000,38500,26500
MachineID,999089,117657,434808,1026470,1001274
ModelID,3157,77,7009,332,4605
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2004
MachineHoursCurrentMeter,68.0,4640.0,2838.0,3486.0,508.0
UsageBand,Low,Low,High,High,Low
fiModelDesc,521D,950FII,226,PC120-6E,310G


In [ ]:

for label, content in df_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_train[label] = pd.Categorical(content).codes+1
for label, content in df_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_train[label] = content.fillna(content.median())

<ipython-input-70-bd1498bfc7cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[label] = pd.Categorical(content).codes+1


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27735 entries, 0 to 32332
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   27735 non-null  int64  
 1   SalePrice                 27735 non-null  int64  
 2   MachineID                 27735 non-null  int64  
 3   ModelID                   27735 non-null  int64  
 4   datasource                27735 non-null  int64  
 5   auctioneerID              27735 non-null  int64  
 6   YearMade                  27735 non-null  int64  
 7   MachineHoursCurrentMeter  27735 non-null  float64
 8   UsageBand                 27735 non-null  int8   
 9   fiModelDesc               27735 non-null  int16  
 10  fiBaseModel               27735 non-null  int16  
 11  fiSecondaryDesc           27735 non-null  int8   
 12  fiModelSeries             27735 non-null  int8   
 13  fiModelDescriptor         27735 non-null  int8   
 14  ProductSize

In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4598 entries, 4 to 31764
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   SalesID                   4598 non-null   int64   
 1   SalePrice                 4598 non-null   int64   
 2   MachineID                 4598 non-null   int64   
 3   ModelID                   4598 non-null   int64   
 4   datasource                4598 non-null   int64   
 5   auctioneerID              4598 non-null   int64   
 6   YearMade                  4598 non-null   int64   
 7   MachineHoursCurrentMeter  4598 non-null   float64 
 8   UsageBand                 4076 non-null   category
 9   fiModelDesc               4598 non-null   category
 10  fiBaseModel               4598 non-null   category
 11  fiSecondaryDesc           3226 non-null   category
 12  fiModelSeries             816 non-null    category
 13  fiModelDescriptor         1287 non-null   category
 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
X_train, y_train = (df_train.drop('SalePrice', axis=1), df_train['SalePrice'])
X_test, y_test = (df_train.drop('SalePrice', axis=1), df_train['SalePrice'])
X_train.shape, X_train.shape, y_train.shape, y_train.shape

((27735, 55), (27735, 55), (27735,), (27735,))

In [ ]:
def rmsle(y_true, y_test):
    return np.sqrt(mean_squared_log_error(y_true, y_test))


In [ ]:
train_preds = model.predict(X_train)


In [ ]:
rmsle(train_preds,y_test)

In [ ]:
df_test=pd.read_csv("/content/Test.csv",parse_dates=['saledate'])

In [ ]:
df_test['month']=df_test.saledate.dt.month
df_test['day']=df_test.saledate.dt.day
df_test['year']=df_test.saledate.dt.year
df_test.drop(['saledate'],axis=1,inplace=True)

In [ ]:
df_test['age']=df_test['year']-df_test['YearMade']

In [ ]:
for label, content in df_test.items():
    if pd.api.types.is_object_dtype(content):
        df_test[label] = content.astype('bool')


In [ ]:
for label, content in df_test.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_test[label] = content.fillna(content.median())

In [ ]:
df_test.info()

In [ ]:
x_test=df_test

In [ ]:
prices=model.predict(x_test)

In [ ]:
df_final=pd.DataFrame()
##df_final=
df_final['price']=prices

In [ ]:
df_final.to_csv("/content/price.csv")